In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2020-05-26 12:10:13--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  6.38MB/s    in 18s     

2020-05-26 12:10:32 (4.56 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [4]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    return data_train, data_test, labels_train, labels_test

In [5]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [6]:
print(train_X[100])
print(train_y[100])

After all, you do not go to an Orson Welles movie to see a nice simple little plot and a burnishing of the image of a happy-ever-after star<br /><br />You go to see theatrically heightened characters locked in conflict against colorful and unusual settings, lighted and scored imaginatively, photographed bravely, and the whole thing peppered with unexpected details of surprise that a wiser and duller director would either avoid or not think of in the first place <br /><br />As usual, as well as directing, Welles wrote the script and he also played the hero  a young Irish seaman who had knocked about the world and seen its evil, but still retained his clear-eyed trust in the goodness of others Unfortunately for him, he reposed this trust in Rita Hayworth, whose cool good looks concealed a gloomy past and murderous inclinations for the future She was married without love, to an impotent, crippled advocate, acted like a malevolent lizard by the brilliant Everett Sloane <br /><br />Th

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [8]:
review_to_words(train_X[100])

['go',
 'orson',
 'well',
 'movi',
 'see',
 'nice',
 'simpl',
 'littl',
 'plot',
 'burnish',
 'imag',
 'happi',
 'ever',
 'star',
 'go',
 'see',
 'theatric',
 'heighten',
 'charact',
 'lock',
 'conflict',
 'color',
 'unusu',
 'set',
 'light',
 'score',
 'imagin',
 'photograph',
 'brave',
 'whole',
 'thing',
 'pepper',
 'unexpect',
 'detail',
 'surpris',
 'wiser',
 'duller',
 'director',
 'would',
 'either',
 'avoid',
 'think',
 'first',
 'place',
 'usual',
 'well',
 'direct',
 'well',
 'wrote',
 'script',
 'also',
 'play',
 'hero',
 'young',
 'irish',
 'seaman',
 'knock',
 'world',
 'seen',
 'evil',
 'still',
 'retain',
 'clear',
 'eye',
 'trust',
 'good',
 'other',
 'unfortun',
 'repos',
 'trust',
 'rita',
 'hayworth',
 'whose',
 'cool',
 'good',
 'look',
 'conceal',
 'gloomi',
 'past',
 'murder',
 'inclin',
 'futur',
 'marri',
 'without',
 'love',
 'impot',
 'crippl',
 'advoc',
 'act',
 'like',
 'malevol',
 'lizard',
 'brilliant',
 'everett',
 'sloan',
 'youth',
 'romantic',
 'underl

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis") 
os.makedirs(cache_dir, exist_ok=True)

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):

    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  
    if cache_data is None:
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


In [11]:
import numpy as np
import re
from collections import Counter

def build_dict(data, vocab_size = 5000):
    word_counts = Counter(np.concatenate( data, axis=0 ))  
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True) 
    word_dict = {} 
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): 
        word_dict[word] = idx + 2                           
        
    return word_dict

In [12]:
word_dict = build_dict(train_X)

In [14]:
data_dir = '../data/pytorch'
if not os.path.exists(data_dir): 
    os.makedirs(data_dir)

In [15]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [16]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0
    INFREQ = 1  
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [17]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [18]:
print(len(train_X[0]))
print(train_X[0])

500
[ 972  148    8  972   18  185 1093  112   70  437   95   64 2349 1206
 1076   31    3    8  178   37  383  763    9   41    1    7  338    9
  350   57   69  234   40    1 1128  213  498   45 1093  648 1500  337
   18  221  112   94  214  498  338   51  540  683    1  972 1009    1
   22  259 1436 1079   94    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  

In [19]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [20]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [21]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [22]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

In [23]:
import torch
import torch.utils.data

train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [24]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            
            output = model(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [25]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.694044578075409
Epoch: 2, BCELoss: 0.6857183098793029
Epoch: 3, BCELoss: 0.678589916229248
Epoch: 4, BCELoss: 0.6706038117408752
Epoch: 5, BCELoss: 0.6607545614242554


In [26]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [27]:
estimator.fit({'training': input_data})

2020-05-26 12:11:30 Starting - Starting the training job...
2020-05-26 12:11:33 Starting - Launching requested ML instances......
2020-05-26 12:12:35 Starting - Preparing the instances for training......
2020-05-26 12:13:58 Downloading - Downloading input data......
2020-05-26 12:14:34 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-26 12:15:03,692 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-26 12:15:03,715 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-26 12:15:03,923 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-26 12:15:04,144 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-05-26 12:15:04,144 sagemaker-containers INFO     Generating setup.cfg
2020-05-26 12:15:04,145 sagemaker-container

In [28]:
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

---------------!

In [30]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [31]:
def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [32]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8504

In [45]:
estimator.delete_endpoint()

In [46]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

In [47]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------!

In [53]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    results = []
    ground = []
   
    for sentiment in ['pos', 'neg']:
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        files_read = 0
        print('Starting ', sentiment, ' files')
 
        for f in files:
            with open(f) as review:
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                review_input = review.read().encode('utf-8')
                results.append(float(predictor.predict(review_input)))

            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [54]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.84

In [57]:
predictor.endpoint

'sagemaker-pytorch-2020-05-26-12-38-23-143'

In [58]:
predictor.delete_endpoint()